# Analyse Data from memory dump or another data set

This assignment is to analyse data. For this I used Seamus's Dowling timeline file base-rd-01-supertimeline.csv which can be found in the Teams files for the course IDR (part of GMIT's certificate in Cybersecurity). I have not uploaded that file to github in this repo though you can find it [here](https://galwaymayoinstitute.sharepoint.com/sites/21-2253897--INCIDENTDETECTIONANDRESPONSE/_layouts/15/Doc.aspx?OR=teams&action=edit&sourcedoc={5AFDCBA0-B5CF-419E-B7AB-63497B65A71B}). You will have to login to GMIT to access it. The file is stored in the same directory that contains this notebook.  


## import the packages

In [1]:
# imports
import pandas as pd # main package for dataframes
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
# magic function to show plots inline in the notebook
import datetime as dt
import matplotlib.ticker as ticker
import seaborn as sns
import re 
from collections import Counter

Firstly, read the file. I looked at the csv headers before loading it to pandas 

In [2]:
# useful links https://github.com/jleaniz/misc/blob/master/timeilne_analysis.ipynb
# cols are date,time,timezone,MACB,source,sourcetype,type,user,host,short,desc,version,filename,inode,notes,format,extra
#filename = 'baserd01-filesystem-timeline.csv' # i was going to use this file but then went with the super timeline for more varied data
filename = 'base-rd-01-supertimeline.csv'
df = pd.read_csv(filename, sep=',')
df

,date,time,timezone,MACB,source,sourcetype,type,user,host,short,desc,version,filename,inode,notes,format,extra
0,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[107 / 0x006b] Strings: ['\Microsoft\Windows\W...,[107 / 0x006b] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
1,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[129 / 0x0081] Strings: ['\Microsoft\Windows\W...,[129 / 0x0081] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
2,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[100 / 0x0064] Strings: ['\Microsoft\Windows\W...,[100 / 0x0064] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
3,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[200 / 0x00c8] Strings: ['\Microsoft\Windows\W...,[200 / 0x00c8] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
4,08/23/2018,00:01:13,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[140 / 0x008c] Strings: ['\Microsoft\Windows\W...,[140 / 0x008c] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185161,09/06/2018,23:58:34,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[1223 / 0x04c7] Strings: ['BND' '3' 'CON\QOS...,[1223 / 0x04c7] Source Name: Microsoft-Windows...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: a677ac5be58f722...
185162,09/06/2018,23:58:34,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[1267 / 0x04f3] Strings: ['BND' '3' 'CON\QOS...,[1267 / 0x04f3] Source Name: Microsoft-Windows...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: a677ac5be58f722...
185163,09/06/2018,23:59:03,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[145 / 0x0091] Strings: ['Enumeration' 'http:...,[145 / 0x0091] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: 6807dc60332085a...
185164,09/06/2018,23:59:03,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[132 / 0x0084] Strings: ['Enumeration'],[132 / 0x0084] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: 6807dc60332085a...


check the shape of the dataframe

In [3]:
df.shape

(185166, 17)

This is a massive file. Hundreds of thousands of rows and 17 columns

In [4]:
type(df) # its a dataframe

pandas.core.frame.DataFrame

In [5]:
df.dtypes # check the types of the columns in the dataframe

date          object
time          object
timezone      object
MACB          object
source        object
sourcetype    object
type          object
user          object
host          object
short         object
desc          object
version        int64
filename      object
inode          int64
notes         object
format        object
extra         object
dtype: object

Almost all the columns are not statistical including the date column 

check the first few values

In [6]:
df.head()

,date,time,timezone,MACB,source,sourcetype,type,user,host,short,desc,version,filename,inode,notes,format,extra
0,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[107 / 0x006b] Strings: ['\Microsoft\Windows\W...,[107 / 0x006b] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
1,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[129 / 0x0081] Strings: ['\Microsoft\Windows\W...,[129 / 0x0081] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
2,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[100 / 0x0064] Strings: ['\Microsoft\Windows\W...,[100 / 0x0064] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
3,08/23/2018,00:01:01,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[200 / 0x00c8] Strings: ['\Microsoft\Windows\W...,[200 / 0x00c8] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
4,08/23/2018,00:01:13,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[140 / 0x008c] Strings: ['\Microsoft\Windows\W...,[140 / 0x008c] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...


check the last few values

In [7]:
df.tail()

,date,time,timezone,MACB,source,sourcetype,type,user,host,short,desc,version,filename,inode,notes,format,extra
185161,09/06/2018,23:58:34,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[1223 / 0x04c7] Strings: ['BND' '3' 'CON\QOS...,[1223 / 0x04c7] Source Name: Microsoft-Windows...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: a677ac5be58f722...
185162,09/06/2018,23:58:34,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[1267 / 0x04f3] Strings: ['BND' '3' 'CON\QOS...,[1267 / 0x04f3] Source Name: Microsoft-Windows...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: a677ac5be58f722...
185163,09/06/2018,23:59:03,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[145 / 0x0091] Strings: ['Enumeration' 'http:...,[145 / 0x0091] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: 6807dc60332085a...
185164,09/06/2018,23:59:03,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[132 / 0x0084] Strings: ['Enumeration'],[132 / 0x0084] Source Name: Microsoft-Windows-...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: 6807dc60332085a...
185165,09/06/2018,23:59:15,UTC,M...,EVT,WinEVTX,Content Modification Time,-,-,[5857 / 0x16e1] Strings: ['WmiPerfInst' '0x00...,[5857 / 0x16e1] Source Name: Microsoft-Windows...,2,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,-,winevtx,recovered: False; sha256_hash: 14f3fb53ec0b7de...


check the column names

In [8]:
df.columns




Index(['date', 'time', 'timezone', 'MACB', 'source', 'sourcetype', 'type',
       'user', 'host', 'short', 'desc', 'version', 'filename', 'inode',
       'notes', 'format', 'extra'],
      dtype='object')

check the unique values in each column, to get a quick overview.

In [9]:
for column in df:
    print(column, "has unique values\n", df[column].unique(), "\n")
    print(column, "has ", df[column].nunique(), " unique values\n")

date has unique values
 ['08/23/2018' '08/24/2018' '08/25/2018' '08/26/2018' '08/27/2018'
 '08/28/2018' '08/29/2018' '08/30/2018' '08/31/2018' '09/01/2018'
 '09/02/2018' '09/03/2018' '09/04/2018' '09/05/2018' '09/06/2018'] 

date has  15  unique values

time has unique values
 ['00:01:01' '00:01:13' '00:03:22' ... '23:58:32' '23:59:03' '23:59:15'] 

time has  34868  unique values

timezone has unique values
 ['UTC'] 

timezone has  1  unique values

MACB has unique values
 ['M...' '....' 'MACB' '.A..' '...B' 'MA.B' 'MAC.' '..CB' '.A.B' '.ACB'
 '..C.' '.AC.' 'MA..' 'M.C.' 'M..B'] 

MACB has  15  unique values

source has unique values
 ['EVT' 'WEBHIST' 'LOG' 'FILE' 'REG' 'PE' 'LNK' 'OLECF' 'META' 'RECBIN'] 

source has  10  unique values

sourcetype has unique values
 ['WinEVTX' 'Chrome Cookies' 'WinPrefetch' 'Mactime Bodyfile' 'UNKNOWN'
 'AppCompatCache Registry Entry' 'Chrome Cache' 'PE Compilation time'
 'Windows Shortcut' 'File entry shell item' 'Task Cache' 'System'
 'Chrome Histor

extra has unique values
 ['recovered: False; sha256_hash: cde410aa7b94ef1002200c891682572d7cdc5edc6fd6e6182708447e8c3c1950; strings_parsed: {}; user_sid: S-1-5-18; xml_string: <Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">-  <System>-    <Provider Name="Microsoft-Windows-TaskScheduler" Guid="{DE7B24EA-73C8-4A09-985D-5BDADCFA9017}"/>-    <EventID>107</EventID>-    <Version>0</Version>-    <Level>4</Level>-    <Task>107</Task>-    <Opcode>0</Opcode>-    <Keywords>0x8000000000000000</Keywords>-    <TimeCreated SystemTime="2018-08-23T00:01:01.046364200Z"/>-    <EventRecordID>187435</EventRecordID>-    <Correlation ActivityID="{6C43A72F-4B21-4071-B619-CE26EF192DC1}"/>-    <Execution ProcessID="1484" ThreadID="1816"/>-    <Channel>Microsoft-Windows-TaskScheduler/Operational</Channel>-    <Computer>base-rd-01.shieldbase.lan</Computer>-    <Security UserID="S-1-5-18"/>-  </System>-  <EventData Name="TimeTriggerEvent">-    <Data Name="TaskName">\\Microsoft\\Windows\\Windo

Drop the columns that don't have interesting stuff - have a quick look first.

In [10]:
todrop=[]
for column in df:
    if (df[column].nunique() == 1):
        print(column)
        todrop.append(column)
df[todrop].head()

        

timezone
user
version
notes


,timezone,user,version,notes
0,UTC,-,2,-
1,UTC,-,2,-
2,UTC,-,2,-
3,UTC,-,2,-
4,UTC,-,2,-


In [11]:
for column in df:
    if (df[column].nunique() == 1):
        df.drop(column,axis=1, inplace =True)
        
df

,date,time,MACB,source,sourcetype,type,host,short,desc,filename,inode,format,extra
0,08/23/2018,00:01:01,M...,EVT,WinEVTX,Content Modification Time,-,[107 / 0x006b] Strings: ['\Microsoft\Windows\W...,[107 / 0x006b] Source Name: Microsoft-Windows-...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
1,08/23/2018,00:01:01,M...,EVT,WinEVTX,Content Modification Time,-,[129 / 0x0081] Strings: ['\Microsoft\Windows\W...,[129 / 0x0081] Source Name: Microsoft-Windows-...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
2,08/23/2018,00:01:01,M...,EVT,WinEVTX,Content Modification Time,-,[100 / 0x0064] Strings: ['\Microsoft\Windows\W...,[100 / 0x0064] Source Name: Microsoft-Windows-...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
3,08/23/2018,00:01:01,M...,EVT,WinEVTX,Content Modification Time,-,[200 / 0x00c8] Strings: ['\Microsoft\Windows\W...,[200 / 0x00c8] Source Name: Microsoft-Windows-...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
4,08/23/2018,00:01:13,M...,EVT,WinEVTX,Content Modification Time,-,[140 / 0x008c] Strings: ['\Microsoft\Windows\W...,[140 / 0x008c] Source Name: Microsoft-Windows-...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: cde410aa7b94ef1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185161,09/06/2018,23:58:34,M...,EVT,WinEVTX,Content Modification Time,-,[1223 / 0x04c7] Strings: ['BND' '3' 'CON\QOS...,[1223 / 0x04c7] Source Name: Microsoft-Windows...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: a677ac5be58f722...
185162,09/06/2018,23:58:34,M...,EVT,WinEVTX,Content Modification Time,-,[1267 / 0x04f3] Strings: ['BND' '3' 'CON\QOS...,[1267 / 0x04f3] Source Name: Microsoft-Windows...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: a677ac5be58f722...
185163,09/06/2018,23:59:03,M...,EVT,WinEVTX,Content Modification Time,-,[145 / 0x0091] Strings: ['Enumeration' 'http:...,[145 / 0x0091] Source Name: Microsoft-Windows-...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: 6807dc60332085a...
185164,09/06/2018,23:59:03,M...,EVT,WinEVTX,Content Modification Time,-,[132 / 0x0084] Strings: ['Enumeration'],[132 / 0x0084] Source Name: Microsoft-Windows-...,OS:E:\C\Windows\system32\winevt\logs\Microsoft...,0,winevtx,recovered: False; sha256_hash: 6807dc60332085a...


check the shape again

In [12]:
df.shape # cols are now 13

(185166, 13)

### Date

In [13]:
#recap
print(df['date'].unique())
print(df['date'].nunique())

['08/23/2018' '08/24/2018' '08/25/2018' '08/26/2018' '08/27/2018'
 '08/28/2018' '08/29/2018' '08/30/2018' '08/31/2018' '09/01/2018'
 '09/02/2018' '09/03/2018' '09/04/2018' '09/05/2018' '09/06/2018']
15


All 15 dates are in August and September 2018. 

### Time

In [14]:
#recap
print(df['time'].unique())
print(df['time'].nunique())

['00:01:01' '00:01:13' '00:03:22' ... '23:58:32' '23:59:03' '23:59:15']
34868


wide range of time values (34868 of them). 24 hour clock used and seconds recorded.

Date and time should be in datetime format so join them

### Make datetime

In [15]:
df['datetime'] = df['date']+"/"+df['time']
df['datetime']

0         08/23/2018/00:01:01
1         08/23/2018/00:01:01
2         08/23/2018/00:01:01
3         08/23/2018/00:01:01
4         08/23/2018/00:01:13
                 ...         
185161    09/06/2018/23:58:34
185162    09/06/2018/23:58:34
185163    09/06/2018/23:59:03
185164    09/06/2018/23:59:03
185165    09/06/2018/23:59:15
Name: datetime, Length: 185166, dtype: object

In [16]:
df['datetime'] = pd.to_datetime(df['datetime'], format="%m/%d/%Y/%H:%M:%S") # format is mm/dd/yyyy/hh:mm:ss

In [17]:
df['datetime'] # now in datetime format

0        2018-08-23 00:01:01
1        2018-08-23 00:01:01
2        2018-08-23 00:01:01
3        2018-08-23 00:01:01
4        2018-08-23 00:01:13
                 ...        
185161   2018-09-06 23:58:34
185162   2018-09-06 23:58:34
185163   2018-09-06 23:59:03
185164   2018-09-06 23:59:03
185165   2018-09-06 23:59:15
Name: datetime, Length: 185166, dtype: datetime64[ns]

In [18]:
df['datetime'].min() # earliest time

Timestamp('2018-08-23 00:01:01')

earliest date is aug 2018 just after midnight

In [19]:
df['datetime'].max() # latest time

Timestamp('2018-09-06 23:59:15')

latest date is sept 2018 just before midnight

The timeframe is 2 weeks of data

In [20]:
df['datetime'].max() - df['datetime'].min()

Timedelta('14 days 23:58:14')

In [21]:
# you can get the various times out with datetime functions if you want to
print("the year is \n",df['datetime'].dt.year)
print("the month is \n", df['datetime'].dt.month)
print("the day is \n",df['datetime'].dt.day) 


the year is 
 0         2018
1         2018
2         2018
3         2018
4         2018
          ... 
185161    2018
185162    2018
185163    2018
185164    2018
185165    2018
Name: datetime, Length: 185166, dtype: int64
the month is 
 0         8
1         8
2         8
3         8
4         8
         ..
185161    9
185162    9
185163    9
185164    9
185165    9
Name: datetime, Length: 185166, dtype: int64
the day is 
 0         23
1         23
2         23
3         23
4         23
          ..
185161     6
185162     6
185163     6
185164     6
185165     6
Name: datetime, Length: 185166, dtype: int64


### MACB

The MACB notation refers to window events for files/directories. See the flyer below. It looks like the '...' refers to no file change. The letter M, A, C, B refers to the file being modified, accessed, changed or created.   
Seamus Dowling's slides on timelining note the following

### Timestamp metadata
* M –Data content last modified
* A –Data content last accessed
* C –Metadata content changed on MFT
* B –Metadata first created (Birth)

![Picure](images/PosterSANS-1024x715.png)

In [22]:
# recap
print(df['MACB'].unique())
print(df['MACB'].nunique())

['M...' '....' 'MACB' '.A..' '...B' 'MA.B' 'MAC.' '..CB' '.A.B' '.ACB'
 '..C.' '.AC.' 'MA..' 'M.C.' 'M..B']
15


There are 15 different combos of MACB. This is useful for finding newly created files and deleted files.

### Source

In [23]:
# recap
print(df.source.unique())
print(df.source.nunique())

['EVT' 'WEBHIST' 'LOG' 'FILE' 'REG' 'PE' 'LNK' 'OLECF' 'META' 'RECBIN']
10


This looks like info regarding to where the file came from e.g events, web history, logs, file, registry, pe?, link, olecf?, meta data, recycling bin. pe is portable executable file and olecf is an embedded object (thanks google).

There are 10 different types of sources of data

### Sourcetype

In [24]:
# recap
print(df.sourcetype.unique())
print(df.sourcetype.nunique())

['WinEVTX' 'Chrome Cookies' 'WinPrefetch' 'Mactime Bodyfile' 'UNKNOWN'
 'AppCompatCache Registry Entry' 'Chrome Cache' 'PE Compilation time'
 'Windows Shortcut' 'File entry shell item' 'Task Cache' 'System'
 'Chrome History' 'UNKNOWN : BagMRU' nan 'Chrome Extensions Autoupdater'
 'UNKNOWN : MRUListEx' 'OLECF Item' 'Chrome Extension Installation'
 'Registry Key: UserAssist' 'Open XML Metadata' 'UNKNOWN : Network Drive'
 'UNKNOWN : Typed URLs' 'UNKNOWN : Run Key' 'UNKNOWN : MRU List'
 'UNKNOWN : RDP Connection' 'Recycle Bin' 'System: Network Connection'
 'Registry Key Shutdown Entry' 'UNKNOWN : Winlogon']
29


This looks like a sub division of source eg. winevtx is probably windows event files and part of event data. Chrome cookies is part of web history  as is Chrome Cache and Chrome History  (I imagine). Do a quick check to see if thats true 

In [25]:
df[['source', 'sourcetype']][(df['source']=='WEBHIST') & ((df['sourcetype']=='Chrome History'))] # all the chrome history source types are source = WEBHIST

,source,sourcetype
17771,WEBHIST,Chrome History
17772,WEBHIST,Chrome History
17849,WEBHIST,Chrome History
17916,WEBHIST,Chrome History
17927,WEBHIST,Chrome History
...,...,...
20402,WEBHIST,Chrome History
20403,WEBHIST,Chrome History
21003,WEBHIST,Chrome History
21004,WEBHIST,Chrome History


In [26]:
df[['source', 'sourcetype']][(df['source']=='WEBHIST') & ((df['sourcetype']=='Chrome Cookies'))] # all the chrome cookies sourcetypes are source = WEBHIST

,source,sourcetype
43,WEBHIST,Chrome Cookies
44,WEBHIST,Chrome Cookies
65,WEBHIST,Chrome Cookies
66,WEBHIST,Chrome Cookies
5309,WEBHIST,Chrome Cookies
...,...,...
173901,WEBHIST,Chrome Cookies
173902,WEBHIST,Chrome Cookies
173903,WEBHIST,Chrome Cookies
173904,WEBHIST,Chrome Cookies


There are 29 source types. I wonder how they brake down by source?

In [29]:
print(df.groupby(['source'])['sourcetype'].value_counts())

source   sourcetype                   
EVT      WinEVTX                          152724
FILE     Mactime Bodyfile                   8394
         File entry shell item               375
LNK      Windows Shortcut                    258
LOG      WinPrefetch                         828
         System                              225
         Chrome Extension Installation        17
         Chrome Extensions Autoupdater         3
         System: Network Connection            1
META     Open XML Metadata                     1
OLECF    OLECF Item                           29
PE       PE Compilation time                   7
RECBIN   Recycle Bin                           1
REG      UNKNOWN                           17080
         UNKNOWN : BagMRU                    194
         Task Cache                          158
         UNKNOWN : MRUListEx                  40
         AppCompatCache Registry Entry        36
         Registry Key: UserAssist             34
         UNKNOWN : MRU List   

All evt are windows eventx artifacts, webhist contains cache, cookie and history from chrome etc. Most rows are windows event logs. 

### Type

In [ ]:
# recap
print(df.type.unique())
print(df.type.nunique())

Type appears to be the readable form of MACB. 

however there are not the same number of unique MACB's as types, so maybe it depends on where MACB code is located.

In [ ]:
df.type.nunique()-df.MACB.nunique()

In [ ]:
df.MACB.unique()

### Short

In [ ]:
#recap 
print(df.short.unique())
print(df.short.nunique())

This is an array with strings that have info. This might  have interesting data. The first array looks like event numbers. The strings do not fully show in the notebook and are truncated (which is annoying- though it crashes if its changed to show). It might be useful to export this column to csv and then read it as a new dataframe with a separator. 

In [ ]:
'''
df['short'].to_csv('short.csv')
df_short = pd.read_csv('short.csv', sep='\[(.*?)\]')
df_short.head() # this does not work yet - if I have time I'll get back to it
'''

### Desc

In [ ]:
#recap 
print(df.desc.unique())
print(df.desc.nunique())


This looks to be the same as short but in a different format and more verbose

In [ ]:
df[['short', 'desc']]

There is extra info in the desc regarding the string e.g. Source Name: Microsoft-Windows-PushNotification-PlatformStrings: desc is more verbose

I checked and 107 is 0x006b in hex so the first part is dec\hex which can be split out. The second part is a string which I'll split out. I'll use short instead of desc as its shorter.

In [ ]:
df_short_strings = df.short.str.split("Strings: ", expand=True).copy()
df_short_strings.columns = ['event_id_dec_hex', 'event_message']
df_short_id_dec = df_short_strings['event_id_dec_hex'].str.split("[", expand=True).copy() 
df_short_id_dec.drop([2, 3], axis =1, inplace=True)
df_short_id_dec = df_short_id_dec[1].str.split("/", expand=True).copy()
df_short_id_dec.drop([2, 3, 4, 5], axis =1, inplace=True)
df_short_id_dec.columns = ['id_dec', 'id_hex']
df_short_id_hex = df_short_id_dec['id_hex'].copy()
df_short_id_dec = df_short_id_dec['id_dec'].copy()
df_short_id_hex = df_short_id_hex.str.split("]", expand=True).copy()
df_short_id_hex.drop([1], axis=1, inplace=True)
df_short_id_hex.columns =['id_hex']
df_short_id_dec.columns = ['id_dec']
df_short_strings.drop('event_id_dec_hex', axis=1, inplace=True)
df_short_strings = df_short_strings.event_message.str.split("[", expand = True).copy()
df_short_strings =df_short_strings.iloc[:,1].str.split("]", expand = True).copy()
df_short_strings.drop([1], axis=1, inplace=True)
df_short_strings.columns=["short string"]
df =pd.concat([df, df_short_id_dec, df_short_id_hex, df_short_strings], axis =1)


check to see if the new columns are there (id_dec, id_hex, short_string)

In [ ]:
df.columns # the new cols id_hex, id_dec and short_string are present

In [ ]:
df.head() # check how it looks

### Filename

In [ ]:
#recap
print(df.filename.unique()[0:10]) # its too long so i'll just print out a few
print(df.filename.nunique())

An array with the path to a filename event view log. Interesting. This could be split pull out file path details.

Lots of unique filenames

### inode

In [ ]:
print(df.inode.unique())
print(df.inode.nunique())


Not sure what to do with that info. 

### Format

In [ ]:
#recap
print(df.format.unique())
print(df.format.nunique())


An array of categories with sub categories. Is it similar to source?

In [ ]:
df.groupby(['source'])['format'].value_counts()

Appears to be more finely tuned than source, source is a category and format is sub category. All evt are winevtx's.

### Extra

In [ ]:
#recap
print(df.extra.unique()[0:5]) # too much info to print
print(df.extra.nunique())

This has lots of potentially useful information

In [ ]:
df.extra.head()

After looking over this it seems that desc has the majority of the data in the dataframe and additional data is in extra.  Its hard to see in jupyter notebook cause of the elipses... (which are also there when viewed in vscode). I'll try and list them out for a look.

In [ ]:
desc_lst = df.desc.to_list() # look at desc list
for item in desc_lst[0:10000:500]: # just look at a sample as file is big
    print("\n", item)

In [ ]:
extra_lst = df.extra.to_list() # look at extra list
for item in extra_lst[0:10000:1000]: # dont print them all out as file is big.
    print("\n", item)

Extra has keys and values. How can i access the keys in his array? try splitting on ';'

In [ ]:
df_extra_split = df.extra.str.split(";", expand=True)


for c in range(5, 101):
    df_extra_split.drop([c], axis=1, inplace=True)

df_extra_split.columns = ['extra_recovered', 'extra_sha256_hash', 'extra_strings_parsed', 'extra_user_sid', 'extra_xml_string']
df_extra_split
df_extra_split.extra_recovered.replace(to_replace="recovered: ", value="", regex=True, inplace=True)
df_extra_split.extra_sha256_hash.replace(to_replace="sha256_hash: ", value="", regex=True, inplace=True)
df_extra_split.extra_strings_parsed.replace(to_replace="strings_parsed: ", value="", regex=True, inplace=True)
df_extra_split.extra_user_sid.replace(to_replace="user_sid: ", value="", regex=True, inplace=True)
df_extra_split.extra_xml_string.replace(to_replace="xml_string: ", value="", regex=True, inplace=True)
df_extra_split



Add the columns to df

In [ ]:
df =pd.concat([df, df_extra_split], axis =1)
df.head()

check if the new cols are there.

In [ ]:
df.columns # the extra_recovered, extra_sha256_hash, extra_stings_parsed, extra_user_sid and extra_xml_string are in the dataframe df

Have a look at the new columns

In [ ]:
for column in df[['extra_recovered', 'extra_sha256_hash', 'extra_strings_parsed', 'extra_user_sid', 'extra_xml_string']]:
    print(df[column].nunique())

In [ ]:
for column in df[['extra_recovered', 'extra_sha256_hash', 'extra_strings_parsed', 'extra_user_sid', 'extra_xml_string']][0:10000:5000]:
    print("\n",column, 'has ', df[column].nunique(), ' values with sample shown below \n ', df[column].unique(), 'v')
    
    # just view a sample as its a big file. 

Look at some statistical info

In [ ]:
df.describe()

This does not produce anything useful here. Only inode has statistical info but its not useful to analyse it that way.

I've added a few cols which changed the shape

In [ ]:
df.shape # quick look at the shape

### revisit filename

Going back to filename - pull out the user from the filename

In [ ]:
filename_users = df.filename.str.split(r"\\")
                                  
print(filename_users.str[3].unique())
df['filename_users'] = filename_users.str[3]
df['filename_users'].unique()

spsql looks like a sql account maybe. The workstation user is tdungan. I don't know what .shieldbase extention is.  

Lets look at spsql user

In [ ]:

spsql_user = df[df['filename_users']=='spsql'] # cant see it in the notebook with the ...
#print(spsql_user)
for u in spsql_user['desc'][0:10000:500].items(): # too much info take sample
    print(u)


Lots of no values stored in keys. Otherwise I'm none the wiser. Lets look at the Temp user.

In [ ]:
temp_user = df[df['filename_users']=='Temp'] # cant see it in the notebook with the ...
print(temp_user)
for u in temp_user['desc'].items():
    print(u)


The PE files are interesting. Lets look at the Public user

In [ ]:
public_user = df[df['filename_users']=='Public'] # cant see it in the notebook with the ...
print(public_user)
for u in public_user['desc'].items():
    print(u)


Public user seems to be for Internet access as its all about Chrome. Lets look at the long string user.

In [ ]:
string_user = df[df['filename_users']=='S-1-5-21-3445421715-2530590580-3149308974-1193'] # cant see it in the notebook with the ...
print(string_user)
for u in string_user['desc'].items():
    print(u)


tdungan is Timothy Dungan and he saved sawaguchi's backup log which is deleted. 

In [ ]:
string_user = df[df['filename_users']=='rsydow-a'] # cant see it in the notebook with the ...
print(string_user)
for u in string_user['desc'].items():
    print(u)


I dont know what rsydow-a is. Lots of Unknown sourcetype. 

### Prefetch

Prefetch is a good place to look for suspicous stuff

In [ ]:
df[['sourcetype', 'filename']][df.sourcetype=='WinPrefetch']

Pull out the prefetch information. For this I split df into a prefetch dataframe for just prefetch info. 

In [ ]:
# trouble with regex bad escape at \C so i'll just remove \
# hide the warnings for now
import warnings 
warnings.filterwarnings('ignore')

myregex = "OS:E:\\C\\Windows\\prefetch\\"

prefetch = df.loc[df['filename'].str.startswith(myregex)] 
prefetch['exe'] =prefetch['filename'].str.replace('\\','').str.replace('OS:E:CWindowsprefetch', '').str.split('-').str[0]
prefetch['pf'] = prefetch['filename'].str.replace('\\','').str.replace('OS:E:CWindowsprefetch', '').str.split('-').str[1] 
prefetch

# i could have filtered by sourcetype instead of using prefetch dataframe



In [ ]:
prefetch['exe'].unique() # look at unique exe's

Items to note include schtasks, powershell, wmiprvse, rdpclip, wsmprovhost, taskkill. Use SAN's filtering and that pdf on excel hunting to look for suspicious items in prefetch data. 

In [ ]:
# https://sansorg.egnyte.com/dl/ZkAyckjFTI
# list of suspicious items taken from SANS Find Evil poster and from this pdf https://www.giac.org/paper/gcih/10588/hunting-log-data-excel/104581
# to just get at.exe use '^at.exe' 
evil_things = ["$C", "Admin$", "psexec.exe", "PsExec", "psexesvc", "^at.exe", "schtasks.exe", "^sc.exe", "wmic.exe", "wmiprvse.exe"
       "scrcons.exe", "mofcomp.exe", "powershell.exe","PowerShell" "wsmprovhost.exe", "find.exe", "ipconfig.exe", "reg.exe", "neti.exe", 
              'tasklist.exe', '^cmd.exe', '^net.exe', 'pe.exe']
# uppercase it as its capitalised in prefetch
evil_things = [e.upper() for e in evil_things]

In [ ]:
prefetch.columns # just checking the column names

In [ ]:
for evil_thing in evil_things:
    if prefetch[prefetch['exe'].str.contains(evil_thing)].empty:
        continue
    else:
        print("Evil found for ", evil_thing, prefetch[['datetime', 'exe']][prefetch['exe'].str.contains(evil_thing)]) 

ipconfig was run at 2018-09-05 11:55:31 - whats that about?

In [ ]:
# to get rid of row ... so i can see things
#pd.set_option('display.max_row',1000) # still not working right :(

In [ ]:
# lets look around the ipconfig execution time
start_time = '2018-09-05 11:55:00' # start a bit before ipconfig time of 2018-09-05 11:55:31
end_time = '2018-09-05 11:56:00' # end a bit after ipconfig time
mask = (df['datetime'] >= start_time) & (df['datetime'] <= end_time) 
df.loc[mask]
    

#### Sample analysis:
wmiprvse.exe was run; spsql user is doing something;  RDP client disconnected warnings - thats remote desktop; and a few seconds later 09/05/2018 11:55:18 RDP client disconnected warning is given. Then cosa clients ran (think thats for mobiles - maybe connecting phone to desktop?) followed by a 'logon office click' to run service monitor; dllhost.exe run 44 times, runtimebroker.exe was executed a lot. Rdp client warning occurred again then ipconfig run. After that the remote desktop connection attempt was tried at event 141380 	09/05/2018 	11:55:54 which succeeded at event 141382 	09/05/2018 	11:55:54 - Lots of Microsoft stuff was scheduelled so it might just be an update or it might not. There was also some game related programmes executed. It would be better to use a SIEM to analyse this type of information.  

In [ ]:
# https://social.technet.microsoft.com/wiki/contents/articles/37870.remote-desktop-client-troubleshooting-disconnect-codes-and-reasons.aspx
# codes for remote desktop connections are listed in the link above

Powershell was run three times in rapid succession on the 2018-09-06 20:30:00 - scheduled tasks were set up at  2018-09-06 20:39:16  possibly for persistence - check out data on 2018-09-06 round 20.30

In [ ]:
start_time = '2018-09-06 20:29:00'
end_time = '2018-09-06 20:50:00'
mask = (df['datetime'] > start_time) & (df['datetime'] <= end_time) 
df.loc[mask]
start_index =df.index[df.index[df['datetime'] > start_time]].min()
end_index =df.index[df.index[df['datetime'] <= end_time]].max()

for e in df.iloc[range(int(start_index), int(end_index)+1)].items():
    print(e)
# still can see what is going on - would be better in a siem here.

In [ ]:
df['filename_users'].unique() # quick look at users again

Look at the number of sourcetypes per filename user

In [ ]:
df[['sourcetype', 'filename_users']].groupby('filename_users').count().sort_values('sourcetype', ascending=False)


Most activity is with system32 followed by spsql then Tim. Identify suspicious items

loop over the other cols for suspicious stuff

In [ ]:
df.sourcetype.unique()

In [ ]:
# loop over some other relevant cols and see if suspicious content mentioned (already got the prefetch done)
found_evil=[]
for column in df[['MACB', 'source', 'sourcetype', 'type', 'short', 'desc', 'filename', 'extra']]: 
    for evil_thing in evil_things:
        if df[df[column].str.contains(evil_thing, na=False)].empty:
            continue
        else:
            print("Evil found for ", evil_thing, df[df[column].str.contains(evil_thing, na=False)]) 
            found_evil.append(df[df[column].str.contains(evil_thing, na=False)])

This lists potential problems so maybe save it?

In [ ]:
found_evil_df = pd.DataFrame(found_evil) 
    
# saving the dataframe 
found_evil_df.to_csv('found_evil.csv') 
print('done')

Another quick look

In [ ]:
print(type(found_evil))
print(len(found_evil))


### MACB revisited

just having a look at type and MACB again

In [ ]:
print(df.type.unique() )
print(df.MACB.unique())

Items with MACB in MACB column indicate newly created files

In [ ]:
# created files
created_files = df.loc[df['MACB'].str.contains('MACB')]
created_files.head()
#created_files['short string']

In [ ]:
created_files.MACB.count()


In [ ]:
len(df[df['MACB']=='MACB']) # quick check

Theres a few hundred newly created files

In [ ]:
created_files.columns
created_files[["date", "MACB", "type"]]

visualise created files

In [ ]:
on_date = '08/28/2018'
created_files_on_date = created_files[created_files['date'] == on_date].copy()
created_files_on_date

Plot created files by date

In [ ]:
#dates = Counter(created_files.date.apply(lambda x: x.strftime('%d-%m-%Y')) )
dates = Counter(created_files.date)
#print(dates)
counts = dates
index = []
data = []

for k,v in counts.items():
    index.append(k)
    data.append(v)
ts = pd.Series(data, index)
figsize=(15, 10)
ts.plot(kind="barh", title="Bar chart of created files per date")
plt.xlabel('Number of created files')
plt.ylabel('date')
plt.show()


Loads of files created on the 28th of August 2018. Look at that date.

In [ ]:
# lots of activity on the 28th
df[['date']]

check_date = r"08/28/2018"
created_files_check_date = created_files[created_files['date']==check_date].copy()

created_files_check_date.head()

In [ ]:
len(created_files_check_date) # 378 created files on that date

Plot the created files on Aug 28th

In [ ]:
dates = Counter(created_files_on_date.time)
#print(dates)
counts = dates
index = []
data = []

for k,v in counts.items():
    index.append(k)
    data.append(v)
ts = pd.Series(data, index)
figsize=(15, 10)
ts.plot(title="Bar chart of created files on date " + on_date)
plt.xlabel('Number of created files')
plt.ylabel('on date '+ on_date)


plt.show()





around 21:40 ish lots of files created. Narrow down the time and have a look.

In [ ]:
start_time = '2018-08-28 21:39:40'
end_time = '2018-08-28 21:42:00'
mask = (df['datetime'] > start_time) & (df['datetime'] <= end_time) 
created_files.loc[mask]
# still can see what is going on - would be better in a siem here.

In [ ]:
created_files.loc[mask]
created_files.loc[mask].filename.unique() # stuff sent to g timeline output - data exfiltration? 

What is that file? It looks like an artifact collector MFTECmd's output. 

### Type revisited

Have a quick look at MACB and type

In [ ]:
df[['MACB', 'type']].groupby('type').count().sort_values("MACB", ascending=False)

In [ ]:
#pd.set_option('display.max_colwidth',1000)# remove elipses to see filename

In [ ]:
## check the file downloaded type
df.filename[df.type=='File Downloaded'].unique() # its web history

What's launch time? 

In [ ]:
df.filename[df.type=='Launch time'].unique() # 

### Plots

Do some plots

### MACB Count

In [ ]:
# plot 
ax = sns.countplot(x="MACB", data=df)
plt.title("Count plot of MACB column")


Most entires are 'M...' - hard to see others so plot without 'M...' for a quick look

In [ ]:
# drop M... to see what the others look look like
sns.countplot(data=df.loc[df['MACB']!="M..."], x='MACB')
plt.title("Count plot of MACB column (without M...)")
plt.xticks(rotation=90)

After 'M...', '..C.' and '.A..' are the most frequent which are last visited time and Metadata Modification Time (see below)

In [ ]:
df.groupby(['type', 'MACB'])['MACB'].count().sort_values(ascending=False)

Plot the counts of 'type' column

In [ ]:
sns.countplot(data=df, x='type')
plt.xticks(rotation=90)
plt.title("Count plot of type column")

content modification time swamps things so plot without it to have a look

In [ ]:
sns.countplot(data=df.loc[df['type']!="Content Modification Time"], x='type')
plt.xticks(rotation=90)
plt.title("Count plot of type column (without Content Modification Time)")

maybe plot the type by date?

In [ ]:
df.type.unique() # checking type values

In [ ]:
# plot 
fig, ax = plt.subplots(figsize=(12,4))
df.groupby(['date'])['type'].count().plot(ax=ax, kind='bar')
plt.title("Count plot of type grouped by date")
plt.xlabel("date");  # custom x label using matplotlib
plt.ylabel("Count of Type");
plt.title('count of type by date')
plt.show


Not sure that is a useful plot. Lots of types used on Aug 28 and 5/6 of Sept. Not much types used on earlier dates in Aug.  

Loop through the columns and get their pivot counts

In [ ]:
for column in df:
    print(column, "\n", df[column].value_counts(), "\n")

### Hist plots

Try histograms

In [ ]:
df.source.hist(bins=10)
plt.title('histogram of counts by source')
plt.xticks(rotation=90)

Loop through certain cols to create various histograms

In [ ]:
for column in df[['date', 'MACB', 'source', 'sourcetype', 'type', 'format']]:
    print(column)
    df[column].hist(bins=10)
    plt.title('histogram of counts by '+column)
    plt.xticks(rotation=90)
    plt.show()

### Countplots

Create some countplots with seaborn

In [ ]:
for column in df[['date', 'MACB', 'source', 'sourcetype', 'type', 'format']]:
    print(column)
    ax = sns.countplot(x=column, data=df)
    plt.title('count plot of '+column)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
df.sort_values(by='date') # python is useful for sorting 

### Conclusion

In conclusion you can do a lot of analysis on log data with python. In particular it gives you an overall feel for the data. This is especially useful when the data is so large as in this file. 